# Overview

This document covers the generation of an event driven analysis of an AIM-format FSAE log using process mining.

## Code Overview

1. Generate events from the log (Ex: Engine RPM high, wide open throttle, max corner).
2. Select an event type of interest and draw a time boundary of +/- seconds around each event occurance.
3. Group all events within each boundary by a common trace ID
4. Perform the process mining
5. Visualize the Performance DF and Markov DF

## Use Cases
- Turn events from a checklist run into DFG graphs.
- Use case 1: (Fault Tree Analysis) Identify how failures cascade across a system (ex: from a minor fault to system failure)
- Use case 2: (Event Tree Analysis) Identify which events are related to each other for system debugging or analysis
- Use case 3: (Test and Verification) Identify if a system can meet a set of performance events, and identify what the deviations are and their behaviors.
- Use Case 4: (Timing analysis) Model the time changes between events, and the 1 sigma deviations to visualize what went through the flow in nominal time, and what traces resulted in longer than nominal.


In [12]:
# Import required packages and setup environment
import pandas as pd
import pandera.pandas as pa

filepath = "./data/raw/FSAE_Endurance_Full.csv"
parsed_filepath = "./data/processed/FSAE_Endurance_Full.csv"

In [13]:
df = pd.read_csv(
    parsed_filepath,
    encoding="latin1",
    low_memory=False,  # Read entire file at once
)

print(df.columns.to_list())

['time.lap_sec', 'distance_km', 'rr.shock_mm', 'rl.shock_mm', 'fl.shock_mm', 'fr.shock_mm', 'acc.lateral_g', 'acc.longitudin_g', 'rear.brake_psi', 'front.brake_psi', 'datalogger.tem_Â°f', 'battery_v', 'f88.rpm_rpm', 'f88.v.speed_mph', 'f88.d.speed_mph', 'f88.speed.fl_mph', 'f88.speed.fr_mph', 'f88.speed.rl_mph', 'f88.speed.rr_mph', 'f88.map1_mbar', 'f88.lambda1_a/f', 'f88.act1_Â°f', 'f88.ect1_Â°f', 'f88.gear_#', 'f88.oil.p1_psi', 'f88.v batt_v', 'f88.fuel.pr1_psi', 'f88.fuel.t_Â°f', 'f88.baro.pr_mbar', 'f88.tps1_%', 'f88.cal.switch_#', 'gps.speed_mph', 'gps.nsat_#', 'gps.latacc_g', 'gps.lonacc_g', 'gps.slope_deg', 'gps.heading_deg', 'gps.gyro_deg/s', 'gps.altitude_m', 'gps.posaccuracy_m', 'fl.shock.pos.zero_mm', 'fr.shock.pos.zero_mm', 'rl.shock.pos.zero_mm', 'rr.shock.pos.zero_mm', 'roll angle_unit', 'fr.roll.gradient_degree', 're.roll.gradient_degree', 'fl.shock.speed_mm/s', 'rr.shock.speed_mm/s', 'rl.shock.speed_mm/s', 'fr.shock.speed_mm/s', 'fl.bumpstop_unit', 'rr.bumpstop_unit', '

# Event conditions

Based on the telemetry columns, here are relevant event conditions for an FSAE team:
1. Lap Events (Fundamental)
Lap Started: lap increments, time.lap_sec resets to ~0
Lap Completed: time.lap_sec reaches max before reset
Sector Crossing: Based on distance_km or aim.distancemeters_m thresholds
2. Driver Performance Events
Braking Events:
Brake Applied: front.brake_psi or rear.brake_psi > threshold (e.g., 50 psi)
Hard Braking: rear.brake_psi > 400 psi AND acc.longitudin_g < -1.0g
Trail Braking: rear.brake_psi decreasing while acc.lateral_g > 0.5g
Brake Released: front.brake_psi < threshold
Cornering Events:
High Lateral Load: acc.lateral_g > 1.2g (aggressive cornering)
Corner Entry: acc.lateral_g increasing + speed decreasing
Corner Apex: acc.lateral_g at local max + gps.gyro_deg/s at max
Corner Exit: acc.lateral_g decreasing + f88.tps1_% increasing
Throttle Events:
Throttle Application: f88.tps1_% > 20%
Full Throttle: f88.tps1_% > 90%
Lift/Coast: f88.tps1_% < 10%
Wheel Spin: f88.d.speed_mph significantly > gps.speed_mph
3. Gear Shift Events
Upshift: f88.gear_# increases
Downshift: f88.gear_# decreases
Shift Under Load: Gear change while f88.tps1_% > 50%
Wrong Gear: f88.rpm_rpm outside optimal range for current gps.speed_mph
4. Suspension/Chassis Events
Bump/Compression:
Bumpstop Hit: fl.bumpstop_unit, fr.bumpstop_unit, etc. = 1 (binary)
Heavy Compression: Shock position (fl.shock_mm, etc.) > 80% travel
Bottoming Event: Max compression rate + bumpstop contact
Roll Event: roll angle_unit or fr.roll.gradient_degree > threshold
Surface Conditions:
Rough Surface: High variance in fl.shock.speed_mm/s or fl.shock.accel_mm/s/s
Jump/Airborne: All shock sensors show rapid extension simultaneously
5. Engine/Powertrain Events
Performance:
Launch: gps.speed_mph 0→moving + f88.tps1_% > 80%
Rev Limiter Hit: f88.rpm_rpm at max sustained value
Overrev: f88.rpm_rpm > safe threshold (e.g., 13,000 rpm)
Lugging: f88.rpm_rpm < 3000 + high load
Fuel System:
Fuel Starvation: f88.lambda1_a/f goes lean (>15) + f88.fuel.pr1_psi drops
Rich Condition: f88.lambda1_a/f < 12.5
High Fuel Flow: fuel flow_cc/min at maximum
6. Temperature Events
Engine Overheating: f88.ect1_°f > 220°F
Oil Overheating: f88.act1_°f > 280°F
Cooling Recovery: Temperature decreasing after peak
7. Pressure/Fluid Events
Low Oil Pressure: f88.oil.p1_psi < 30 psi (critical)
Oil Pressure Spike: run.oil.pres.hi_psi or load.oil.pres.hi_psi exceeds safe limit
Low Fuel Pressure: f88.fuel.pr1_psi < threshold
8. Electrical Events
Low Battery Voltage: battery_v or f88.v batt_v < 12.0V
Voltage Spike: battery_v > 15.0V
GPS Lock Lost: gps.nsat_# < 4
GPS Lock Acquired: gps.nsat_# >= 4
9. Calibration/Mode Events
Calibration Switch Change: f88.cal.switch_# changes value (e.g., rain mode, aggressive mode)
Map Change: f88.map1_mbar threshold changes suggest different tuning
10. Failure/Warning Events
Wheel Lockup: Individual wheel speed (f88.speed.fl_mph, etc.) drops to 0 while others moving
Loss of Traction: Rear wheel speeds >> front wheel speeds
Sensor Anomaly: Any sensor reading NaN, out of physical bounds
Data Logging Issue: cycle time_ms spikes (data acquisition lag)
11. Track Position Events
Straight Section: Low acc.lateral_g + high gps.speed_mph
Technical Section: High frequency f88.gear_# changes
Elevation Change: gps.slope_deg > threshold or gps.elevation_cm changing rapidly
12. Comparative/Session Events
Fastest Sector: Compare time.lap_sec at sector markers across laps
Consistency Check: Lap time variance
Setup Change: Between-run comparisons (different sessions in time.session_sec)


In [14]:
"""Event Extraction from FSAE Telemetry Data

This module extracts discrete events from continuous telemetry data based on
threshold conditions and state changes.

Considerations: Raw threshold-based methods on noisy telemetry can generate
spurious events. Consider adding hysteresis (different thresholds for entering vs. exiting 
a state) or minimum dwell times to avoid event flooding. Threshold-based detection works 
well for known failure modes but struggles with gradual drift or context-dependent anomalies.

"""

import numpy as np
import pandas as pd
from typing import List, Dict, Tuple


class EventExtractor:
    """Extract events from FSAE telemetry dataframe based on defined conditions."""
    
    def __init__(self, df: pd.DataFrame):
        """Initialize with telemetry dataframe.
        
        Args:
            df: Telemetry dataframe with all sensor channels
        """
        self.df = df
        self.events = []
        
    def detect_threshold_events(
        self, 
        column: str, 
        threshold: float, 
        condition: str,
        event_name: str,
        min_duration_rows: int = 1
    ) -> pd.DataFrame:
        """Detect events where a column crosses a threshold.
        
        Args:
            column: Column name to monitor
            threshold: Threshold value
            condition: Comparison operator ('>', '<', '>=', '<=', '==')
            event_name: Name of the event
            min_duration_rows: Minimum number of consecutive rows to confirm event
            
        Returns:
            DataFrame with columns: timestamp, event_name, value, lap
        """
        # Create boolean mask based on condition
        if condition == '>':
            mask = self.df[column] > threshold
        elif condition == '<':
            mask = self.df[column] < threshold
        elif condition == '>=':
            mask = self.df[column] >= threshold
        elif condition == '<=':
            mask = self.df[column] <= threshold
        elif condition == '==':
            mask = self.df[column] == threshold
        else:
            raise ValueError(f"Unknown condition: {condition}")
        
        # Find rising edges (transitions from False to True)
        rising_edge = mask & ~mask.shift(1).fillna(False)
        
        # Filter by minimum duration if specified
        if min_duration_rows > 1:
            # Check if condition stays true for min_duration_rows
            duration_check = pd.Series(False, index=self.df.index)
            for i in range(min_duration_rows):
                duration_check |= mask.shift(-i).fillna(False)
            rising_edge = rising_edge & duration_check
        
        # Extract events
        event_indices = self.df[rising_edge].index
        events_df = pd.DataFrame({
            'timestamp': self.df.loc[event_indices, 'time.absolute'],
            'activity': event_name,
            'value': self.df.loc[event_indices, column],
            'lap': self.df.loc[event_indices, 'lap'],
            'session_time': self.df.loc[event_indices, 'time.session_sec']
        })
        
        return events_df.reset_index(drop=True)
    
    def detect_state_change_events(
        self,
        column: str,
        event_name_prefix: str,
        ignore_nan: bool = True
    ) -> pd.DataFrame:
        """Detect when a column value changes (e.g., gear shifts, calibration changes).
        
        Args:
            column: Column name to monitor
            event_name_prefix: Prefix for event name (will append old->new values)
            ignore_nan: Whether to ignore NaN values
            
        Returns:
            DataFrame with event details
        """
        # Find where values change
        if ignore_nan:
            value_changed = (self.df[column] != self.df[column].shift(1)) & \
                           self.df[column].notna() & \
                           self.df[column].shift(1).notna()
        else:
            value_changed = self.df[column] != self.df[column].shift(1)
        
        event_indices = self.df[value_changed].index
        
        events_df = pd.DataFrame({
            'timestamp': self.df.loc[event_indices, 'time.absolute'],
            'activity': [
                f"{event_name_prefix} {self.df.loc[idx-1, column]:.0f}->{self.df.loc[idx, column]:.0f}"
                if idx > self.df.index[0] else f"{event_name_prefix} Start"
                for idx in event_indices
            ],
            'value': self.df.loc[event_indices, column],
            'lap': self.df.loc[event_indices, 'lap'],
            'session_time': self.df.loc[event_indices, 'time.session_sec']
        })
        
        return events_df.reset_index(drop=True)
    
    def detect_combined_condition_events(
        self,
        conditions: List[Tuple[str, str, float]],
        event_name: str,
        mode: str = 'all'
    ) -> pd.DataFrame:
        """Detect events based on multiple conditions.
        
        Args:
            conditions: List of (column, operator, threshold) tuples
            event_name: Name of the event
            mode: 'all' (AND) or 'any' (OR) for combining conditions
            
        Returns:
            DataFrame with event details
        """
        masks = []
        for column, operator, threshold in conditions:
            if operator == '>':
                masks.append(self.df[column] > threshold)
            elif operator == '<':
                masks.append(self.df[column] < threshold)
            elif operator == '>=':
                masks.append(self.df[column] >= threshold)
            elif operator == '<=':
                masks.append(self.df[column] <= threshold)
            elif operator == '==':
                masks.append(self.df[column] == threshold)
        
        # Combine masks
        if mode == 'all':
            combined_mask = pd.Series(True, index=self.df.index)
            for mask in masks:
                combined_mask &= mask
        else:  # 'any'
            combined_mask = pd.Series(False, index=self.df.index)
            for mask in masks:
                combined_mask |= mask
        
        # Find rising edges
        rising_edge = combined_mask & ~combined_mask.shift(1).fillna(False)
        event_indices = self.df[rising_edge].index
        
        events_df = pd.DataFrame({
            'timestamp': self.df.loc[event_indices, 'time.absolute'],
            'activity': event_name,
            'value': None,
            'lap': self.df.loc[event_indices, 'lap'],
            'session_time': self.df.loc[event_indices, 'time.session_sec']
        })
        
        return events_df.reset_index(drop=True)
    
    def detect_local_extrema_events(
        self,
        column: str,
        event_name_max: str,
        event_name_min: str,
        window_size: int = 10,
        prominence: float = 0.1
    ) -> pd.DataFrame:
        """Detect local maxima and minima (e.g., corner apex).
        
        Args:
            column: Column to analyze
            event_name_max: Name for maximum events
            event_name_min: Name for minimum events
            window_size: Size of window for local comparison
            prominence: Minimum prominence (difference from neighbors)
            
        Returns:
            DataFrame with event details
        """
        from scipy.signal import find_peaks
        
        values = self.df[column].fillna(0).values
        
        # Find peaks (maxima)
        peaks_max, _ = find_peaks(values, distance=window_size, prominence=prominence)
        # Find valleys (minima)
        peaks_min, _ = find_peaks(-values, distance=window_size, prominence=prominence)
        
        # Create events for maxima
        events_max = pd.DataFrame({
            'timestamp': self.df.iloc[peaks_max]['time.absolute'].values,
            'activity': event_name_max,
            'value': self.df.iloc[peaks_max][column].values,
            'lap': self.df.iloc[peaks_max]['lap'].values,
            'session_time': self.df.iloc[peaks_max]['time.session_sec'].values
        })
        
        # Create events for minima
        events_min = pd.DataFrame({
            'timestamp': self.df.iloc[peaks_min]['time.absolute'].values,
            'activity': event_name_min,
            'value': self.df.iloc[peaks_min][column].values,
            'lap': self.df.iloc[peaks_min]['lap'].values,
            'session_time': self.df.iloc[peaks_min]['time.session_sec'].values
        })
        
        return pd.concat([events_max, events_min], ignore_index=True).sort_values('timestamp')
    

print("EventExtractor class defined successfully")

EventExtractor class defined successfully


In [15]:
# Initialize the event extractor
extractor = EventExtractor(df)

print(f"Event extractor initialized with {len(df)} telemetry rows")
print(f"Time range: {df['time.session_sec'].min():.2f}s to {df['time.session_sec'].max():.2f}s")
print(f"Number of laps: {df['lap'].max()}")

Event extractor initialized with 190587 telemetry rows
Time range: 0.01s to 1905.56s
Number of laps: 22


# Extract Events

Now we'll extract various types of events from the telemetry data using the EventExtractor class.

In [16]:
"""Extract all event types from telemetry data."""

all_events = []

# 1. LAP EVENTS
print("Extracting lap events...")
lap_events = extractor.detect_state_change_events(
    column='lap',
    event_name_prefix='Lap'
)
all_events.append(lap_events)
print(f"  Found {len(lap_events)} lap transitions")

# 2. GEAR SHIFT EVENTS
print("Extracting gear shift events...")
gear_events = extractor.detect_state_change_events(
    column='f88.gear_#',
    event_name_prefix='Gear Shift'
)
all_events.append(gear_events)
print(f"  Found {len(gear_events)} gear shifts")

# 3. BRAKING EVENTS
print("Extracting braking events...")
brake_applied = extractor.detect_threshold_events(
    column='front.brake_psi',
    threshold=50,
    condition='>',
    event_name='Brake Applied',
    min_duration_rows=3
)
all_events.append(brake_applied)
print(f"  Found {len(brake_applied)} brake applications")

hard_braking = extractor.detect_combined_condition_events(
    conditions=[
        ('rear.brake_psi', '>', 400),
        ('acc.longitudin_g', '<', -1.0)
    ],
    event_name='Hard Braking',
    mode='all'
)
all_events.append(hard_braking)
print(f"  Found {len(hard_braking)} hard braking events")

# 4. THROTTLE EVENTS
print("Extracting throttle events...")
full_throttle = extractor.detect_threshold_events(
    column='f88.tps1_%',
    threshold=90,
    condition='>',
    event_name='Full Throttle',
    min_duration_rows=5
)
all_events.append(full_throttle)
print(f"  Found {len(full_throttle)} full throttle events")

# 5. CORNERING EVENTS - Local maxima in lateral acceleration
print("Extracting cornering events...")
corner_events = extractor.detect_local_extrema_events(
    column='acc.lateral_g',
    event_name_max='Corner Apex (Left)',
    event_name_min='Corner Apex (Right)',
    window_size=20,
    prominence=0.3
)
all_events.append(corner_events)
print(f"  Found {len(corner_events)} corner apex events")

# 6. HIGH LATERAL LOAD EVENTS
print("Extracting high lateral load events...")
high_lateral = extractor.detect_threshold_events(
    column='acc.lateral_g',
    threshold=1.2,
    condition='>',
    event_name='High Lateral Load',
    min_duration_rows=5
)
all_events.append(high_lateral)
print(f"  Found {len(high_lateral)} high lateral load events")

# 7. BUMPSTOP EVENTS
print("Extracting suspension bumpstop events...")
for corner in ['fl', 'fr', 'rl', 'rr']:
    bumpstop = extractor.detect_threshold_events(
        column=f'{corner}.bumpstop_unit',
        threshold=0.5,
        condition='>',
        event_name=f'Bumpstop Hit ({corner.upper()})',
        min_duration_rows=1
    )
    if len(bumpstop) > 0:
        all_events.append(bumpstop)
        print(f"  Found {len(bumpstop)} bumpstop hits on {corner.upper()}")

# 8. ENGINE EVENTS
print("Extracting engine events...")
high_rpm = extractor.detect_threshold_events(
    column='f88.rpm_rpm',
    threshold=11000,
    condition='>',
    event_name='High RPM',
    min_duration_rows=10
)
all_events.append(high_rpm)
print(f"  Found {len(high_rpm)} high RPM events")

# 9. LOW OIL PRESSURE WARNING
print("Extracting oil pressure events...")
low_oil_pressure = extractor.detect_threshold_events(
    column='f88.oil.p1_psi',
    threshold=30,
    condition='<',
    event_name='Low Oil Pressure Warning',
    min_duration_rows=5
)
if len(low_oil_pressure) > 0:
    all_events.append(low_oil_pressure)
    print(f"  Found {len(low_oil_pressure)} low oil pressure warnings")

# 10. GPS EVENTS
print("Extracting GPS events...")
gps_lock_lost = extractor.detect_threshold_events(
    column='gps.nsat_#',
    threshold=4,
    condition='<',
    event_name='GPS Lock Lost',
    min_duration_rows=10
)
if len(gps_lock_lost) > 0:
    all_events.append(gps_lock_lost)
    print(f"  Found {len(gps_lock_lost)} GPS lock lost events")

print(f"\n{'='*60}")
print(f"Total event categories: {len(all_events)}")
print(f"Total events extracted: {sum(len(e) for e in all_events)}")

Extracting lap events...
  Found 21 lap transitions
Extracting gear shift events...
  Found 1702 gear shifts
Extracting braking events...
  Found 373 brake applications
  Found 0 hard braking events
Extracting throttle events...
  Found 63 full throttle events
Extracting cornering events...


/tmp/ipykernel_421671/3459389395.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rising_edge = mask & ~mask.shift(1).fillna(False)
/tmp/ipykernel_421671/3459389395.py:72: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  duration_check |= mask.shift(-i).fillna(False)
/tmp/ipykernel_421671/3459389395.py:167: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcas

  Found 4122 corner apex events
Extracting high lateral load events...
  Found 1377 high lateral load events
Extracting suspension bumpstop events...
  Found 19 bumpstop hits on FL
  Found 19 bumpstop hits on FR
  Found 19 bumpstop hits on RL
  Found 19 bumpstop hits on RR
Extracting engine events...
  Found 4 high RPM events
Extracting oil pressure events...
  Found 461 low oil pressure warnings
Extracting GPS events...

Total event categories: 13
Total events extracted: 8199


# Combine Events into Event Log

Merge all extracted events into a single event dataframe suitable for process mining.

In [17]:
# Combine all events into a single dataframe
events_df = pd.concat(all_events, ignore_index=True)

# Sort by timestamp
events_df = events_df.sort_values('timestamp').reset_index(drop=True)

# Add an event_id column
events_df.insert(0, 'event_id', range(1, len(events_df) + 1))

# Add case_id (use lap number as case identifier)
events_df['case_id'] = 'Lap_' + events_df['lap'].astype(int).astype(str)

# Reorder columns for clarity
events_df = events_df[['event_id', 'case_id', 'timestamp', 'activity', 'lap', 'session_time', 'value']]

event_log_path = "./data/processed/FSAE_Event_Log.csv"
events_df.to_csv(event_log_path, index=False)

print(f"Combined Event Log Statistics:")
print(f"{'='*60}")
print(f"Total events: {len(events_df)}")
print(f"Unique activities: {events_df['activity'].nunique()}")
print(f"Unique cases (laps): {events_df['case_id'].nunique()}")
print(f"Time span: {events_df['timestamp'].min()} to {events_df['timestamp'].max()}")
print(f"\nEvent distribution by activity type:")
print(events_df['activity'].value_counts().head(15))
print(f"\nFirst 10 events:")
events_df.head(10)

Combined Event Log Statistics:
Total events: 8199
Unique activities: 52
Unique cases (laps): 22
Time span: 2016-05-14T16:27:26.010000Z to 2016-05-14T16:59:09.000000Z

Event distribution by activity type:
activity
Corner Apex (Right)         2070
Corner Apex (Left)          2052
High Lateral Load           1377
Low Oil Pressure Warning     461
Brake Applied                373
Gear Shift 3->3              322
Gear Shift 4->4              284
Gear Shift 2->2              178
Gear Shift 5->5              163
Gear Shift 4->3              112
Gear Shift 3->4              112
Gear Shift 4->5               86
Gear Shift 5->4               84
Gear Shift 3->2               76
Gear Shift 2->3               76
Name: count, dtype: int64

First 10 events:


/tmp/ipykernel_421671/108663020.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events_df = pd.concat(all_events, ignore_index=True)


,event_id,case_id,timestamp,activity,lap,session_time,value
0,1,Lap_1,2016-05-14T16:27:26.010000Z,Bumpstop Hit (FL),1,0.01,5.0
1,2,Lap_1,2016-05-14T16:27:26.010000Z,Bumpstop Hit (RL),1,0.01,5.0
2,3,Lap_1,2016-05-14T16:27:26.010000Z,Bumpstop Hit (RR),1,0.01,5.0
3,4,Lap_1,2016-05-14T16:27:26.010000Z,Low Oil Pressure Warning,1,0.01,0.0
4,5,Lap_1,2016-05-14T16:27:26.010000Z,Bumpstop Hit (FR),1,0.01,5.0
5,6,Lap_1,2016-05-14T16:27:27.930000Z,Bumpstop Hit (FR),1,1.93,2.0
6,7,Lap_1,2016-05-14T16:27:27.930000Z,Bumpstop Hit (RL),1,1.93,2.0
7,8,Lap_1,2016-05-14T16:27:27.930000Z,Bumpstop Hit (FL),1,1.93,2.0
8,9,Lap_1,2016-05-14T16:27:27.930000Z,Bumpstop Hit (RR),1,1.93,2.0
9,10,Lap_1,2016-05-14T16:27:44.090000Z,Gear Shift 0->0,1,18.09,0.4
